In [ ]:
import numpy as np
import pandas as pd
import torch
import matplotlib.pyplot as plt

In [ ]:
# Load the dataset
cath_df = pd.read_csv("./data/cath_moments.tsv", sep='\t', header=None) # For Train
ecod_df = pd.read_csv("./data/ecod_moments.tsv", sep='\t', header=None) # For eval

In [ ]:
cath_info = cath_df.info()

In [ ]:
ecod_info = ecod_df.info()

In [ ]:
# Summarize shapes, classes, and a preview
cath_summary = {
    "shape": cath_df.shape,
    "unique_classes": cath_df[0].nunique(),
    "class_distribution": cath_df[0].value_counts(),
    "head": cath_df.head()
}

ecod_summary = {
    "shape": ecod_df.shape,
    "unique_classes": ecod_df[0].nunique(),
    "class_distribution": ecod_df[0].value_counts(),
    "head": ecod_df.head()
}

In [ ]:
print(cath_summary)

In [ ]:
print(ecod_summary)

In [ ]:
a = [1.2,3.4,5.6],
b = [2.4,3.2,4.5]

z_ab = abs(a[i]-b[i])

2*(1.2-2.4)/1+abs(1.2)+abs(2.4)

Vector in the original dimenstion

30 proteins 
30*29/2-> pairs of proteins

In [ ]:
cath_df.iloc[0]

In [ ]:
for i in range(0,2):
    print(type(cath_df.iloc[0,i]))

In [ ]:
# Plot class distribution for CATH
cath_class_counts = cath_df[0].value_counts()
plt.figure(figsize=(12, 4))
plt.hist(cath_class_counts, bins=30, edgecolor='black')
plt.title('Class Frequency Distribution in CATH Dataset')
plt.xlabel('Number of proteins per class')
plt.ylabel('Number of classes')
plt.grid(True)
plt.tight_layout()
plt.show()

# Plot class distribution for ECOD
ecod_class_counts = ecod_df[0].value_counts()
plt.figure(figsize=(12, 4))
plt.hist(ecod_class_counts, bins=20, edgecolor='black')
plt.title('Class Frequency Distribution in ECOD Dataset')
plt.xlabel('Number of proteins per class')
plt.ylabel('Number of classes')
plt.grid(True)
plt.tight_layout()
plt.show()


### Visualizing Class Imbalance

Each protein in the dataset is associated with a structural class label (column 0), which identifies its 3D(1D in this case) shape category. Understanding how many proteins fall into each class is critical because the dataset is not balanced—some classes have many proteins, while others have very few.

To quantify this, we use `value_counts()` to compute the frequency of each class label and visualize the distribution with a histogram.

#### Why this is important:
During training, we will create pairs of proteins to determine structural similarity. If a particular class contains many proteins, it will generate significantly more pairs, which may bias the model toward frequently occurring classes. This can lead to overfitting and poor generalization, especially on underrepresented classes.

By plotting the class frequency distribution:
- We confirm whether class imbalance is present.
- We motivate the need for sampling strategies such as `WeightedRandomSampler` to correct for this imbalance during training.

This analysis is a key step in understanding the structure of the data and informing how we design the training process.


In [ ]:
import sys
import numpy as np

print("Python version:", sys.version)
print("NumPy version:", np.__version__)


In [1]:
import pandas as pd
import torch
import importlib
import train
import dataset
import time
import os
importlib.reload(train)
importlib.reload(dataset)
import random

random.seed(42)


from train import train_model, test_model_on_ecod

[INFO] Using 9 CPU threads
[INFO] Using 9 CPU threads


In [2]:
# Load datasets
cath_df = pd.read_csv("./data/cath_moments.tsv", sep='\t', header=None).dropna(axis=1)
ecod_df = pd.read_csv("./data/ecod_moments.tsv", sep='\t', header=None).dropna(axis=1)

print(f"CATH: {cath_df.shape}, ECOD: {ecod_df.shape}")
features,labels=None,None


CATH: (2685, 3923), ECOD: (761, 3923)


In [3]:
num_proteins = 2685

In [4]:
# import pandas as pd
# from cache_utils import cache_pairwise_data

# df = pd.read_csv("./data/cath_moments.tsv", sep="\t", header=None).dropna(axis=1)


# cache_dir = "./cache/cath_"+str(num_proteins)
# merge_dir = cache_dir+"/cath_merged.pkl"

# # if not os.path.exists(merge_dir):
# #     cache_pairwise_data(df.head(num_proteins), cache_dir=cache_dir, buffer_limit_mb=100)

In [ ]:
# from cache_utils import load_cached_parts,load_and_merge_parts
# from dataset import ProteinPairDataset

# tic = time.time_ns()
# # Step 1: Load and merge buffered part_*.pkl files
# features, labels = load_cached_parts(cache_dir, max_threads=16)

# # #Parallel Load all the parts
# # features, labels = load_and_merge_parts(
# #     cache_dir=cache_dir,
# #     save_path=merge_dir, 
# #     max_threads=16
# # )

# tac = time.time_ns()
# print("Loaded in ",(tac-tic)/(10**6),"ms")

In [ ]:
# from cache_utils import load_cached_parts

# # Load cached features/labels 
# features, labels = load_cached_parts("./cache/cath_"+str(num_proteins))

# # Check dimensions
# print(f"[INFO] Loaded {len(features)} pairs with shape {features[0].shape}")

In [5]:
print("Expected pairs:",num_proteins*(num_proteins-1)//2)

Expected pairs: 3603270


In [ ]:
from weight_strategy import inverse_class_weighting
input_dim = cath_df.shape[1]-1
streaming = True
lr = 1e-4

if streaming==False:
    print("-"*20, "CACHE","-"*20)
    model = train_model(features=features, labels=labels, hidden_dim=32, input_dim=input_dim, streaming=streaming,batch_size=128,num_epochs=30,val_split=0.1,lr=lr)
else:
    print("-"*20, "OTG","-"*20)
    model = train_model(protein_df=cath_df.head(num_proteins), hidden_dim=32, input_dim=input_dim, streaming=streaming,batch_size=128,num_epochs=30,val_split=0.1,lr=lr)

In [ ]:
from weight_strategy import inverse_class_weighting
input_dim = cath_df.shape[1]-1
streaming = True

if streaming==False:
    print("-"*20, "CACHE","-"*20)
    model = train_model(features=features, labels=labels, hidden_dim=64, input_dim=input_dim, streaming=streaming,batch_size=128,num_epochs=30,val_split=0.1,lr=lr)
else:
    print("-"*20, "OTG","-"*20)
    model = train_model(protein_df=cath_df.head(num_proteins), hidden_dim=64, input_dim=input_dim, streaming=streaming,batch_size=128,num_epochs=30,val_split=0.1,lr=lr)

In [ ]:
from weight_strategy import inverse_class_weighting
input_dim = cath_df.shape[1]-1
streaming = True

if streaming==False:
    print("-"*20, "CACHE","-"*20)
    model = train_model(features=features, labels=labels, hidden_dim=128, input_dim=input_dim, streaming=streaming,batch_size=128,num_epochs=30,val_split=0.1,lr=lr)
else:
    print("-"*20, "OTG","-"*20)
    model = train_model(protein_df=cath_df.head(num_proteins), hidden_dim=128, input_dim=input_dim, streaming=streaming,batch_size=128,num_epochs=30,val_split=0.1,lr=lr)

In [ ]:
from weight_strategy import inverse_class_weighting
input_dim = cath_df.shape[1]-1
streaming = True

if streaming==False:
    print("-"*20, "CACHE","-"*20)
    model = train_model(features=features, labels=labels, hidden_dim=256, input_dim=input_dim, streaming=streaming,batch_size=128,num_epochs=30,val_split=0.1,lr=lr)
else:
    print("-"*20, "OTG","-"*20)
    model = train_model(protein_df=cath_df.head(num_proteins), hidden_dim=256, input_dim=input_dim, streaming=streaming,batch_size=128,num_epochs=30,val_split=0.1,lr=lr)

In [ ]:
from weight_strategy import inverse_class_weighting
input_dim = cath_df.shape[1]-1
streaming = True

if streaming==False:
    print("-"*20, "CACHE","-"*20)
    model = train_model(features=features, labels=labels, hidden_dim=512, input_dim=input_dim, streaming=streaming,batch_size=128,num_epochs=30,val_split=0.1,lr=lr)
else:
    print("-"*20, "OTG","-"*20)
    model = train_model(protein_df=cath_df.head(num_proteins), hidden_dim=512, input_dim=input_dim, streaming=streaming,batch_size=128,num_epochs=30,val_split=0.1,lr=lr)

In [8]:
from weight_strategy import inverse_class_weighting
input_dim = cath_df.shape[1]-1
streaming = True

lr = 0.000005
if streaming==False:
    print("-"*20, "CACHE","-"*20)
    model = train_model(features=features, labels=labels, hidden_dim=1024, input_dim=input_dim, streaming=streaming,batch_size=128,num_epochs=30,val_split=0.1,lr=lr)
else:
    print("-"*20, "OTG","-"*20)
    model = train_model(protein_df=cath_df.head(num_proteins), hidden_dim=1024, input_dim=input_dim, streaming=streaming,batch_size=128,num_epochs=30,val_split=0.1,lr=lr)

-------------------- OTG --------------------
Training on: cpu (CPU)
[TensorBoard] Logging to: tensorboard_logs/baseline_h1024_bs128_lr5e-06_ep30_20250330-062545
[INFO] Loading Dataloader using streaming : True
[INFO] Creating dataloaders... streaming=True
[INFO] Initializing Streaming Dataset from DataFrame of size 2685
[INFO] Streaming V2 init done in 0.57 seconds
[INFO] Creating traindata with 3242943
[INFO] Creating valdata with 360327
Training model (hidden_dim=1024) for 30 epochs...


Epoch 1/30:   0%|          | 0/25336 [00:00<?, ?it/s]

Epoch 1/30 | Train Loss: 3770.6268 | Val Loss: 208.8888 | ROC AUC: 0.988 | PR AUC: 0.611 | MCC: 0.317


Epoch 2/30:   0%|          | 0/25336 [00:00<?, ?it/s]

Epoch 2/30 | Train Loss: 1626.6882 | Val Loss: 133.7798 | ROC AUC: 0.988 | PR AUC: 0.623 | MCC: 0.372


...

Epoch 28/30 | Train Loss: 75.0835 | Val Loss: 39.8620 | ROC AUC: 0.988 | PR AUC: 0.645 | MCC: 0.565


Epoch 29/30:   0%|          | 0/25336 [00:00<?, ?it/s]

Epoch 29/30 | Train Loss: 73.7308 | Val Loss: 37.5907 | ROC AUC: 0.987 | PR AUC: 0.644 | MCC: 0.580


Epoch 30/30:   0%|          | 0/25336 [00:00<?, ?it/s]

Epoch 30/30 | Train Loss: 72.1046 | Val Loss: 37.2721 | ROC AUC: 0.988 | PR AUC: 0.646 | MCC: 0.578


In [10]:
from weight_strategy import inverse_class_weighting
input_dim = cath_df.shape[1]-1
streaming = True

lr = 1e-5
if streaming==False:
    print("-"*20, "CACHE","-"*20)
    model = train_model(features=features, labels=labels, hidden_dim=1024, input_dim=input_dim, streaming=streaming,batch_size=128,num_epochs=30,val_split=0.1,lr=lr)
else:
    print("-"*20, "OTG","-"*20)
    model = train_model(protein_df=cath_df.head(num_proteins), hidden_dim=1024, input_dim=input_dim, streaming=streaming,batch_size=128,num_epochs=30,val_split=0.1,lr=lr)

-------------------- OTG --------------------
Training on: cpu (CPU)
[TensorBoard] Logging to: tensorboard_logs/baseline_h1024_bs128_lr1e-05_ep30_20250330-133020
[INFO] Loading Dataloader using streaming : True
[INFO] Creating dataloaders... streaming=True
[INFO] Initializing Streaming Dataset from DataFrame of size 2685
[INFO] Streaming V2 init done in 0.89 seconds
[INFO] Creating traindata with 3242943
[INFO] Creating valdata with 360327
Training model (hidden_dim=1024) for 30 epochs...


Epoch 1/30:   0%|          | 0/25336 [00:00<?, ?it/s]

Epoch 1/30 | Train Loss: 2746.5221 | Val Loss: 138.3194 | ROC AUC: 0.989 | PR AUC: 0.625 | MCC: 0.371


Epoch 2/30:   0%|          | 0/25336 [00:00<?, ?it/s]

Epoch 2/30 | Train Loss: 852.9638 | Val Loss: 87.3493 | ROC AUC: 0.988 | PR AUC: 0.640 | MCC: 0.428


...

Epoch 28/30 | Train Loss: 83.7393 | Val Loss: 47.6503 | ROC AUC: 0.985 | PR AUC: 0.620 | MCC: 0.520


Epoch 29/30:   0%|          | 0/25336 [00:00<?, ?it/s]

Epoch 29/30 | Train Loss: 84.8638 | Val Loss: 41.8626 | ROC AUC: 0.987 | PR AUC: 0.627 | MCC: 0.548


Epoch 30/30:   0%|          | 0/25336 [00:00<?, ?it/s]

Epoch 30/30 | Train Loss: 83.7484 | Val Loss: 48.9844 | ROC AUC: 0.988 | PR AUC: 0.643 | MCC: 0.520


In [ ]:
#Load the best model
import torch
from model import ProteinClassifier
input_dim = cath_df.shape[1]-1
# Define your model architecture (must match the saved model)
model = ProteinClassifier(hidden_dim=64, input_dim=input_dim)

# Load saved weights
model.load_state_dict(torch.load("./modelData/baseline_h64_bs128_lr0.001_ep20_20250328-041027_best.pt", map_location="cpu"))

# Set model to evaluation mode
model.eval()

In [11]:
test_model_on_ecod(model, ecod_df)

[INFO] Initializing Streaming Dataset from DataFrame of size 761
[INFO] Streaming init done in 0.10 seconds
[FINAL TEST on ECOD] Loss: 431.4965 | ROC AUC: 0.966 | PR AUC: 0.782 | MCC: 0.705


(431.49647467000403, 0.9660356894615169, 0.782256605396797, 0.704535367872464)

In [ ]:
import pandas as pd
import torch
from torch.utils.data import DataLoader
from dataset import StreamingProteinPairDataset, StreamingProteinPairDatasetV2
import time
import tracemalloc  # optional: for memory profiling

# Load a manageable subset for testing
df = pd.read_csv("data/cath_moments.tsv", sep='\t', header=None).dropna(axis=1).head(1000)

BATCH_SIZE = 64

def profile_loader(dataset_class, name=""):
    print(f"\n--- Profiling {name} ---")

    # Start memory profiling
    tracemalloc.start()
    start = time.time_ns()
    dataset = dataset_class(df)
    init_time_ns = time.time_ns() - start
    current, peak = tracemalloc.get_traced_memory()
    tracemalloc.stop()

    print(f"[{name}] Init time: {init_time_ns / 1e6:.2f} ms")
    print(f"[{name}] Peak memory usage: {peak / (1024**2):.2f} MB")

    # Profile batch loading
    loader = DataLoader(dataset, batch_size=BATCH_SIZE, num_workers=4)
    batch_times_ns = []
    for i, (x, y) in enumerate(loader):
        if i >= 5:
            break
        t0 = time.time_ns()
        _ = x.shape, y.shape  # simulate some access
        t1 = time.time_ns()
        batch_times_ns.append(t1 - t0)

    if batch_times_ns:
        avg_batch_time_ms = sum(batch_times_ns) / len(batch_times_ns) 
        print(f"[{name}] Avg batch load time (first 5): {avg_batch_time_ms:.2f} ms")
    else:
        print(f"[{name}] No batches loaded.")

# Run profiling for both
profile_loader(StreamingProteinPairDataset, "Streaming v1")
profile_loader(StreamingProteinPairDatasetV2, "Streaming v2")